In [ ]:
import urllib.request, json 
import pandas as pd
import mysql.connector

In [ ]:
min_summons_number = 0

In [ ]:
def convert24(str1):
     
    # Checking if last two elements of time
    # is AM and first two elements are 12
    if str1[-2:] == "AM" and str1[:2] == "12":
        return "00" + str1[2:-2]
         
    # remove the AM    
    elif str1[-2:] == "AM":
        return str1[:-2]
     
    # Checking if last two elements of time
    # is PM and first two elements are 12
    elif str1[-2:] == "PM" and str1[:2] == "12":
        return str1[:-2]
         
    else:
         
        # add 12 to hours and remove PM
        return str(int(str1[:2]) + 12) + str1[2:8]

In [ ]:
def converttotime(str1):
    #print(str1[-1])
    str2 = ""
    str2 = str1[-5:-3]+":"+str1[-3:-1]+":00"
    if str1[-1] == 'A':
        str2 = str2+" AM"
    else: 
        str2 = str2+" PM"
    return str2

In [ ]:
df = pd.read_csv('web_scrape.csv')
USER = df.loc[0,'Column 2']
PASSWORD = df.loc[0,'Column 3']
PAGINATION = int(df.loc[1,'Column 3'])

In [ ]:
api_url = "https://data.cityofnewyork.us/resource/pvqr-7yc4.json?$limit=100&$offset="

In [ ]:
def into_database(data):
    for i in range(0,len(data)):
        try:
            mydb = mysql.connector.connect(
              host="localhost",
              user=USER,
              password=PASSWORD,
              database="dmdw"
            )
            summons_number = int(data[i]['summons_number'])
            violation_code = int(data[i]['violation_code'])
            date = data[i]['issue_date'][0:10] + " " +convert24(converttotime(data[i]['violation_time']))
            violation_location = int(data[i]['violation_location'])
            sql = "insert into ticket values("+str(summons_number)+","+str(violation_code)+",'"+date+"',"+str(violation_location)+")"
            mycursor = mydb.cursor()
            mycursor.execute(sql)
            mydb.commit()
            mycursor.close()
            mydb.close()
            #print(sql)
        except Exception as e: print(e)

In [ ]:
i = 1294336
while True:
    i = i+99
    temp = api_url + str(i)
    print(temp)
    try:
        with urllib.request.urlopen(temp) as url:
            data = json.loads(url.read().decode())
        into_database(data)
    except Exception as e: print(e)


# mydb = mysql.connector.connect(
  host="localhost",
  user=USER,
  password=PASSWORD,
  database="dmdw"
)
sql = 'delete from ticket'
mycursor = mydb.cursor()
mycursor.execute(sql)
mydb.commit()
mycursor.close()
mydb.close()